### 13 November 2019
# Calculate recovery of ClinVar pathogenic/likely-pathogenic variants in RefSeq exonic regions (GRCh37) by slop
### by Pavlos Bousounis
***Last updated 11/14/2019***
___

### Import modules

In [293]:
from datetime import datetime
import glob
from natsort import natsorted
import numpy as np
import os
import pandas as pd
import pathlib
import pybedtools
from pybedtools import BedTool
import re

### Set working directory

In [512]:
basedir = '/Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data/'
os.chdir(basedir)

### Save working directory and date

In [296]:
# get current working directory
basedir = os.getcwd()
# get today's date as YYYY-MM-DD
today = datetime.today().strftime('%Y-%m-%d')

print('Currently in ' + basedir + '\n')
print('Today is ' + today)

Currently in /Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data

Today is 2019-11-14


### Define function ***bed_overlap()***
___

In [311]:
def bed_overlap(ref_bed_filepath, test_bed_filepath, dir_out='.'):
    
    """ Given two bed files, ref_bed and test_bed, perform bedtools intersect -c to return the number of 
    test_bed regions that overlap any regions in the ref_bed file. Returns the ref_bed file with a column 
    of overlap counts """
    
    cwd = os.getcwd()
    
    # specify the reference bed file
    ref_bedtool = BedTool(ref_bed_filepath)
    prfx_ref = ref_bed_filepath.split('/')[-1]
    prfx_ref = prfx_ref.split('.')[0]
    
    # specify the new ClinVar bed file
    test_bedtool = BedTool(test_bed_filepath)
    prfx_test = test_bed_filepath.split('/')[-1]
    prfx_test = prfx_test.split('.')[0]

    # specify name/path of output bed file
    bed_out = dir_out + '/{}_IN_{}.bed'.format(prfx_test, prfx_ref)
    
    # run bedtools intersect to get all test_bed regions NOT found in ref_bed (-v option)
    ref_in_test = test_bedtool.intersect(b=ref_bedtool, c=True)
    
    return(ref_in_test)

## Run bedtools slop on RefSeq bed 

#### For ***n*** in {5, 10, 15, .., 150}:
    extend each input bed region by n bases on each end and save as bed to disk
___

In [306]:
# create/designate slop file output directory
slop_dir_out = os.path.join((today + "_RefSeqGRCh37_exon_slop_bed"))
pathlib.Path(slop_dir_out).mkdir(exist_ok=True)

In [309]:
# create the RefSeq bedtool
refseq_bed_filepath = '2019-11-08_RefSeq-GRCh37_latest_genomicGFF3.bed'
refseq_bed = pd.read_csv(refseq_bed_filepath, sep='\t', header=None, low_memory=False)
refseq_bed[0] = 'chr' + refseq_bed.loc[:, 0]    

# create bedtool from dataframe
refseq_bedtool = BedTool.from_dataframe(refseq_bed)

# save headless refseq bed
refseq_bed_out = '2019-11-08_RefSeq-GRCh37_latest_genomicGFF3_headless.bed'
refseq_bed.to_csv(refseq_bed_out, sep='\t', header=None, index=False)

In [310]:
# retrieve hg19 chromosome sizes for pybedtools slop
hg19_genome = pybedtools.helpers.get_chromsizes_from_ucsc(genome='hg19', saveas='data/hg19.genome')

In [17]:
# for each N in {0, 5, 10, .., 155} (non-inclusive) run bedtools slop adding N bases to each end of each region
for i in list(range(5, 155, 5)):
    
    # create the output filename
    file_out = os.path.join(os.getcwd(), slop_dir_out, '{}_RefSeqGRCh37_slop{}.bed'.format(today, i))
    
    # perform slop: add 'i' bases to each end of each region
    refseq_bedtool.slop(g=hg19_genome, b=i).saveas(file_out)
    
    # check file save
    if os.path.isfile(os.path.join(os.getcwd(), file_out)):
        print('\nSuccess! File saved:\n{}\n'.format(file_out))


Success! File saved:
/Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data/2019-11-13_RefSeqGRCh37_exon_slop_bed/2019-11-13_RefSeqGRCh37_slop5.bed


Success! File saved:
/Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data/2019-11-13_RefSeqGRCh37_exon_slop_bed/2019-11-13_RefSeqGRCh37_slop10.bed


Success! File saved:
/Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data/2019-11-13_RefSeqGRCh37_exon_slop_bed/2019-11-13_RefSeqGRCh37_slop15.bed


Success! File saved:
/Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data/2019-11-13_RefSeqGRCh37_exon_slop_bed/2019-11-13_RefSeqGRCh37_slop20.bed


Success! File saved:
/Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data/2019-11-13_RefSeqGRCh37_exon_slop_bed/2019-11-13_RefSeqGRCh37_slop25.bed


Success! File saved:
/Users/pbousou

### Get the ClinVar variant overlap counts for each RefSeq exonic region

In [524]:
# the clinvar bed file to intersect
bed_a_file = '2019-11-07-GRCh37_path-likely_path_headless.bed'

# list of slopped refseq bed files to intersect
slop_files = glob.glob('2019-11-13_RefSeqGRCh37_exon_slop_bed/*RefSeqGRCh37_slop*.bed')

# create/specify output directory
slop_int_out = os.path.join(('../output/' + today + "_RefSeqGRCh37_ClinVar_slop-intersect_bed"))
pathlib.Path(slop_int_out).mkdir(exist_ok=True)

# create empty dict 
for i in slop_files:
    slop_num = re.findall(r'\d+', i)[-1]
    fname = '{}_RefSeqGRCh37_ClinVar_recovery_{}.bed'.format(today, slop_num)
    bed_overlap(bed_a_file, i, slop_int_out)

Success!
File saved to: 
/Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data/../output/2019-11-14_RefSeqGRCh37_ClinVar_slop-intersect_bed/2019-11-13_RefSeqGRCh37_slop45_IN_2019-11-07-GRCh37_path-likely_path_headless.bed.

Success!
File saved to: 
/Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data/../output/2019-11-14_RefSeqGRCh37_ClinVar_slop-intersect_bed/2019-11-13_RefSeqGRCh37_slop50_IN_2019-11-07-GRCh37_path-likely_path_headless.bed.

Success!
File saved to: 
/Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data/../output/2019-11-14_RefSeqGRCh37_ClinVar_slop-intersect_bed/2019-11-13_RefSeqGRCh37_slop85_IN_2019-11-07-GRCh37_path-likely_path_headless.bed.

Success!
File saved to: 
/Users/pbousounis/Experiments/2019-10-29_hg19mod/RefSeq-ClinVar_GRCh37_slop_region_recovery/data/../output/2019-11-14_RefSeqGRCh37_ClinVar_slop-intersect_bed/2019-11-13_RefSe

### Concatenate bed_overlap slop files 

In [525]:
# list overlapped slop files
ovl_slop_files = glob.glob(slop_int_out + '/*RefSeqGRCh37_slop*IN*GRCh37_path-likely_path_headless.bed')

# read in the files and concatenate
bedcols = ['chr', 'start', 'end', 'name', 'exon_recovery']
df = pd.DataFrame(columns=bedcols)
for file in ovl_slop_files:
    
    # import file as dataframe
    tmp = pd.read_csv(file, sep='\t', header=None, names=bedcols)
    
    # extract transcript IDs and gene names to their own columns
    tmp['tx_id'] = tmp['name'].str.extract(r'(\w+-)(N(M|R)_\d+)')[1]
    tmp['gene'] = tmp['name'].str.split('-').str[0]
    
    # add column with slop file ID
    tmp['slop_len'] = re.findall(r'slop\d+', file)[0]
    
    # Filter: remove rows (regions) with no ClinVar overlap
    tmp = tmp[tmp.exon_recovery != 0]
    
    # append to df
    df = df.append(tmp, sort=False)
    del(tmp)

In [331]:
# get transcript recovery by slop distance
df['tx_slop_recovery'] = df['exon_recovery'].groupby([df['tx_id'], df['slop_len']]).transform('sum')

In [ ]:
# get transcript recovery by slop distance ##CHECK LABEL exon_recovery
df['tx_slop_recovery'] = df['exon_recovery'].groupby([df['tx_id'], df['slop_len']]).transform('sum')

In [334]:
# spread df by slop length
slop_df = df.pivot_table(index='tx_id', columns='slop_len', values='tx_slop_recovery')
slop_df = slop_df[natsorted(slop_df.columns)].add_suffix('_recovery')

In [ ]:
# spread df by slop length
slop_df = df.pivot_table(index='tx_id', columns='slop_len', values='tx_slop_recovery')
slop_df = slop_df[natsorted(slop_df.columns)].add_suffix('_recovery')

In [337]:
print('\n')
slop_df.info()
print('\n')
slop_df.head()



<class 'pandas.core.frame.DataFrame'>
Index: 14197 entries, NM_000016 to NR_164148
Data columns (total 30 columns):
slop5_recovery      13937 non-null float64
slop10_recovery     13963 non-null float64
slop15_recovery     13977 non-null float64
slop20_recovery     13998 non-null float64
slop25_recovery     14006 non-null float64
slop30_recovery     14018 non-null float64
slop35_recovery     14028 non-null float64
slop40_recovery     14033 non-null float64
slop45_recovery     14036 non-null float64
slop50_recovery     14040 non-null float64
slop55_recovery     14048 non-null float64
slop60_recovery     14055 non-null float64
slop65_recovery     14064 non-null float64
slop70_recovery     14066 non-null float64
slop75_recovery     14067 non-null float64
slop80_recovery     14087 non-null float64
slop85_recovery     14097 non-null float64
slop90_recovery     14098 non-null float64
slop95_recovery     14107 non-null float64
slop100_recovery    14117 non-null float64
slop105_recovery    14

slop_len,slop5_recovery,slop10_recovery,slop15_recovery,slop20_recovery,slop25_recovery,slop30_recovery,slop35_recovery,slop40_recovery,slop45_recovery,slop50_recovery,...,slop105_recovery,slop110_recovery,slop115_recovery,slop120_recovery,slop125_recovery,slop130_recovery,slop135_recovery,slop140_recovery,slop145_recovery,slop150_recovery
tx_id,,,,,,,,,,,,,,,,,,,,,
NM_000016,73.0,74.0,74.0,74.0,74.0,74.0,74.0,74.0,74.0,74.0,...,74.0,74.0,78.0,79.0,79.0,79.0,79.0,79.0,79.0,80.0
NM_000017,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,...,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0
NM_000018,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,97.0,...,129.0,136.0,139.0,146.0,150.0,155.0,158.0,165.0,171.0,178.0
NM_000019,28.0,29.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,...,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0
NM_000020,129.0,129.0,129.0,129.0,129.0,129.0,129.0,129.0,129.0,129.0,...,129.0,129.0,129.0,129.0,129.0,129.0,129.0,129.0,129.0,129.0


In [535]:
# import the transcript percent recovery table
tx_rec_file = '2019-11-12_RefSeq-ClinVar_transcript-recovery.tsv'
tx_rec_cols = ['tx_id', 'gene', 'slop0_recovery', 'gene_recovery', 'slop0_recovery%']
tx_rec = pd.read_csv(tx_rec_file, sep='\t', skiprows=1, names=tx_rec_cols)
tx_rec.head()

,tx_id,gene,slop0_recovery,gene_recovery,slop0_recovery%
0,NM_005101,ISG15,3,3.0,100.000000
1,NM_001305275,AGRN,15,15.0,100.000000
2,NM_198576,AGRN,15,15.0,100.000000
3,NM_001364727,AGRN,13,15.0,86.666667
4,NM_080605,B3GALT6,9,9.0,100.000000


In [351]:
# add gene_recovery column to slop_df
slop_tx = slop_df.merge(tx_rec[['tx_id', 'gene_recovery']], how='left', on='tx_id')

In [ ]:
# add gene_recovery column to slop_df
slop_tx = slop_df.merge(tx_rec[['tx_id', 'gene_recovery']], how='left', on='tx_id')

In [352]:
# calculate percent recovery for each slop length
for col in slop_tx.loc[:, slop_tx.columns.str.contains('slop')].columns:
    new_col = col.split('_')[0] + '_recovery%'
    slop_tx[new_col] = slop_tx[col] / slop_tx['gene_recovery'] * 100

In [365]:
tmp_slop_table = pd.merge(tx_rec[['tx_id', 'gene', 'slop0_recovery', 'slop0_recovery%']], slop_tx, how='left', on='tx_id')
tmp_slop_table.head()

,tx_id,gene,slop0_recovery,slop0_recovery%,slop5_recovery,slop10_recovery,slop15_recovery,slop20_recovery,slop25_recovery,slop30_recovery,...,slop105_recovery%,slop110_recovery%,slop115_recovery%,slop120_recovery%,slop125_recovery%,slop130_recovery%,slop135_recovery%,slop140_recovery%,slop145_recovery%,slop150_recovery%
0,NM_005101,ISG15,3,100.000000,3.0,3.0,3.0,3.0,3.0,3.0,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,NM_001305275,AGRN,15,100.000000,15.0,15.0,15.0,15.0,15.0,15.0,...,100.000000,100.000000,100.000000,100.000000,106.666667,113.333333,113.333333,113.333333,133.333333,133.333333
2,NM_198576,AGRN,15,100.000000,15.0,15.0,15.0,15.0,15.0,15.0,...,100.000000,100.000000,100.000000,100.000000,106.666667,113.333333,113.333333,113.333333,133.333333,133.333333
3,NM_001364727,AGRN,13,86.666667,13.0,13.0,13.0,13.0,13.0,13.0,...,86.666667,86.666667,86.666667,86.666667,93.333333,100.000000,100.000000,100.000000,120.000000,120.000000
4,NM_080605,B3GALT6,9,100.000000,9.0,9.0,9.0,9.0,9.0,9.0,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [434]:
# reorder columns
slop_cols_list = ['tx_id', 'gene'] + natsorted(tmp_slop_table.loc[:, tmp_slop_table.columns.str.contains('slop')])
slop_table = tmp_slop_table[slop_cols_list]
slop_table.head()

,tx_id,gene,slop0_recovery,slop0_recovery%,slop5_recovery,slop5_recovery%,slop10_recovery,slop10_recovery%,slop15_recovery,slop15_recovery%,...,slop130_recovery,slop130_recovery%,slop135_recovery,slop135_recovery%,slop140_recovery,slop140_recovery%,slop145_recovery,slop145_recovery%,slop150_recovery,slop150_recovery%
0,NM_005101,ISG15,3,100.000000,3.0,100.000000,3.0,100.000000,3.0,100.000000,...,3.0,100.000000,3.0,100.000000,3.0,100.000000,3.0,100.000000,3.0,100.000000
1,NM_001305275,AGRN,15,100.000000,15.0,100.000000,15.0,100.000000,15.0,100.000000,...,17.0,113.333333,17.0,113.333333,17.0,113.333333,20.0,133.333333,20.0,133.333333
2,NM_198576,AGRN,15,100.000000,15.0,100.000000,15.0,100.000000,15.0,100.000000,...,17.0,113.333333,17.0,113.333333,17.0,113.333333,20.0,133.333333,20.0,133.333333
3,NM_001364727,AGRN,13,86.666667,13.0,86.666667,13.0,86.666667,13.0,86.666667,...,15.0,100.000000,15.0,100.000000,15.0,100.000000,18.0,120.000000,18.0,120.000000
4,NM_080605,B3GALT6,9,100.000000,9.0,100.000000,9.0,100.000000,9.0,100.000000,...,9.0,100.000000,9.0,100.000000,9.0,100.000000,9.0,100.000000,9.0,100.000000


In [460]:
# round _recovery% columns to two decimal places
slop_table[natsorted(tmp_slop_table.loc[:, tmp_slop_table.columns.str.contains('_recovery%')])] = slop_table.loc[:, slop_table.columns.str.contains('_recovery%')].round(2)

# convert recovery columns to integers
slop_table[natsorted(tmp_slop_table.loc[:, tmp_slop_table.columns.str.contains('slop.+_recovery$')])] = slop_table[natsorted(tmp_slop_table.loc[:, tmp_slop_table.columns.str.contains('slop.+_recovery$')])].astype('int')

/opt/anaconda3/envs/biopython/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


## Create new column of top slop distances (<= 100% recovery)

In [493]:
# subset columns
slop_tmp = slop_table.set_index('tx_id')
slop_perc_cols = slop_tmp.loc[:, slop_tmp.columns.str.contains('_recovery%')]

slop_perc_cols = slop_perc_cols[slop_perc_cols <= 100]

# create the top slop column (= slop distance with maximum recovery <= 100%)
# returns first (lowest) slop distance meeting above criteria
slop_perc_cols['top_slop'] = slop_perc_cols.idxmax(axis=1)

In [494]:
# reconstitute the full slop table
# trim _recovery% columns from slop_table 
slop_rec = slop_table.loc[:, slop_table.columns.str.contains('_recovery$') | slop_table.columns.str.contains('tx_id') | slop_table.columns.str.contains('gene')]

# add top_slop column
slop_table_top = slop_rec.merge(slop_perc_cols, on='tx_id', how='left', suffixes=(False, False))

# add gene_recovery column
gene_recovery_df = tx_rec.iloc[:, [0,3]]
slop_master = slop_table_top.merge(gene_recovery_df, on='tx_id', how='left', suffixes=(False, False))

# reorder columns
slop_master_cols = slop_master.columns.tolist()

slop_cols = ['tx_id', 'gene', 'top_slop', 'gene_recovery'] + natsorted(slop_master.loc[:, slop_master.columns.str.contains('slop.+recovery')])
slop_master_table = slop_master[slop_cols]

slop_master_table.head()

,tx_id,gene,gene_recovery,slop0_recovery,slop5_recovery,slop10_recovery,slop15_recovery,slop20_recovery,slop25_recovery,slop30_recovery,...,slop110_recovery%,slop115_recovery%,slop120_recovery%,slop125_recovery%,slop130_recovery%,slop135_recovery%,slop140_recovery%,slop145_recovery%,slop150_recovery%,top_slop
0,NM_005101,ISG15,3.0,3,3,3,3,3,3,3,...,100.00,100.00,100.00,100.00,100.0,100.0,100.0,100.0,100.0,slop0_recovery%
1,NM_001305275,AGRN,15.0,15,15,15,15,15,15,15,...,100.00,100.00,100.00,NaN,NaN,NaN,NaN,NaN,NaN,slop0_recovery%
2,NM_198576,AGRN,15.0,15,15,15,15,15,15,15,...,100.00,100.00,100.00,NaN,NaN,NaN,NaN,NaN,NaN,slop0_recovery%
3,NM_001364727,AGRN,15.0,13,13,13,13,13,13,13,...,86.67,86.67,86.67,93.33,100.0,100.0,100.0,NaN,NaN,slop130_recovery%
4,NM_080605,B3GALT6,9.0,9,9,9,9,9,9,9,...,100.00,100.00,100.00,100.00,100.0,100.0,100.0,100.0,100.0,slop0_recovery%


## Save slop recovery table

In [513]:
slop_master_table_out = '../output/' + today + '_RefSeq-ClinVar_GRCh37_slop-recovery-master.tsv'
slop_master_table.to_csv(slop_master_table_out, sep='\t', index=False)

if os.path.isfile(slop_master_table_out):
    print('\nSuccess! File saved to:\n{}'.format(tmp2_out))


Success! File saved to:
../output/2019-11-13_RefSeq-ClinVar_GRCh37_slop-recovery-master.tsv
